You need to make sure QuPath.exe is in usr/bin or else [check Paquo documentation](https://paquo.readthedocs.io/en/latest/configuration.html#configuration) about making it accessible to Python. Make sure to pip install paquo and rasterio. This takes about 20 seconds.

In [1]:
from paquo.projects import QuPathProject
import numpy as np
# from tifffile import imread
import rasterio
from rasterio import features
import shapely
from shapely.geometry import Point, Polygon

In [36]:
def mask_to_polygons_layer_new(mask):
	all_polygons = [None] * mask.max()
	for shape, value in features.shapes(mask.astype(np.int32), mask=(mask >0), transform=rasterio.Affine(1.0, 0, 0, 0, 1.0, 0)):
		all_polygons[int(value - 1)] = shapely.geometry.shape(shape)
	return all_polygons

In [21]:
# nuclei_mask = imread("./DAPI-mask.tif")
# cytoplasms_mask = imread("./cytoplasms.tif")
nuclei_mask = np.load('./mask.npy')
cytoplasms_mask = np.load('./expanded.npy')

In [37]:
nuclei = mask_to_polygons_layer_new(nuclei_mask)
cytoplasms = mask_to_polygons_layer_new(cytoplasms_mask)

In [ ]:
# test_measurements1 = np.random.randint(0, 100, size=(len(nuclei), 3)).astype(float) # need to make real measurements later, 3 is number of columns
# test_measurements2 = np.random.randint(0, 100, size=(len(cytoplasms), 3)).astype(float) # need to make real measurements later, 3 is number of columns

In [42]:
with QuPathProject("./folder-name/project.qpproj", mode = "x") as qp:
	entry = qp.add_image('Brain.ome.tif') # replace with your image
	hierarchy = entry.hierarchy
	with hierarchy.no_autoflush():
		for i, nucleus in enumerate(nuclei):
			if not isinstance(nucleus, shapely.geometry.base.BaseGeometry): continue
			else:
				detection = hierarchy.add_detection(nucleus)
				detection.name = f"Nucleus {i}"
		for i, cytoplasm in enumerate(cytoplasms):
			if not isinstance(cytoplasm, shapely.geometry.base.BaseGeometry): continue
			else:
				detection = hierarchy.add_detection(cytoplasm)
				detection.name = f"Cytoplasm {i}"

No matching downsample for thumbnail! This might take a long time...


15:51:08.281 [main] [DEBUG] qupath.lib.projects.DefaultProject - Writing project to /Volumes/HP P500/Final G6r1a QuPath/detections-regex-5/project.qpproj.tmp
15:51:08.284 [main] [DEBUG] qupath.lib.projects.DefaultProject - Backing up existing project to /Volumes/HP P500/Final G6r1a QuPath/detections-regex-5/project.qpproj.backup
15:51:08.284 [main] [DEBUG] qupath.lib.projects.DefaultProject - Renaming project to /Volumes/HP P500/Final G6r1a QuPath/detections-regex-5/project.qpproj
